# Предобработка твитов

Передоставлен большой датасет с твитами. Нужно научиться определять, какие твиты негативной тональности, а какие — позитивной. Чтобы решить эту задачу, из открытого репозитория [DeepPavlov](https://docs.deeppavlov.ai/en/master/features/models/bert.html) возьмём модель _RuBERT_, обученную на разговорном русскоязычном корпусе. 

Решим эту задачу на **PyTorch**. Она применяется в задачах обработки естественного текста и компьютерного зрения. А нам нужна для работы с моделями типа _BERT_.

In [4]:
# импорт библиотек
import torch
import transformers
import numpy as np
import pandas as pd

У _RuBERT_ есть собственный токенизатор. Это инструмент, который разбивает и преобразует исходные тексты в список токенов, которые есть, например, в словаре _RuBERT_. Лемматизация не требуется.

Инициализируем токенизатор как объект класса `BertTokenizer()`. Передадим ему аргумент `vocab_file` — это файл со словарём, на котором обучалась модель. Он может быть, например, в текстовом формате (txt).

In [19]:
# чтение датасета
df = pd.read_csv('/datasets/tweets.csv')

# инициализируем токенизатор
tokenizer = transformers.BertTokenizer(vocab_file='/datasets/ds_bert/vocab.txt')

Преобразуем текст в номера токенов из словаря методом encode() (англ. «закодировать»)

Для корректной работы модели мы указали аргумент `add_special_tokens` (англ. «добавить специальные токены»), равный `True`. Это значит, что к любому преобразуемому тексту добавляется токен начала (101) и токен конца текста (102).

Применим метод `padding`, чтобы после токенизации длины исходных текстов в корпусе были равными. Только при таком условии будет работать модель _BERT_. Пусть стандартной длиной вектора n будет длина наибольшего во всём датасете вектора. Остальные векторы дополним нулями.

И поясним модели, что нули не несут значимой информации. Это нужно для компоненты модели, которая называется `attention`. Отбросим эти токены и «создадим маску» для действительно важных токенов, то есть укажем нулевые и не нулевые значения.

In [23]:
# 1ый вариант обработки датасета
tokens_list = []
attention_mask_list = []
for i in range(len(df)):
    # токенизируем текст
    vector = tokenizer.encode(df.loc[i,'text'], add_special_tokens=True)
    
    # применим padding к векторам
    n = 133
    padded = vector + [0]*(n - len(vector))
    
    # создадим маску для важных токенов
    attention_mask = np.where(padded != 0, 1, 0)
    
    # сохранение токена и маски важности
    tokens_list.append(padded)
    attention_mask_list.append(attention_mask)
    
print(len(attention_mask_list))

5000


In [25]:
# 2ой вариант обработки датасета
# токенизируем текст
tokens = df['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

# применим padding к векторам
tokens_padded = tokens.apply(lambda x: x + [0]*(n - len(x)))

# создадим маску для важных токенов
attention_mask = tokens_padded.apply(lambda x: np.where(x != 0, 1, 0))    

print(len(attention_mask_list))

5000
